In [ ]:
pip install -U langchain requests langchain_mistralai

In [ ]:
import requests
import os
from dotenv import load_dotenv
from langchain.agents import AgentExecutor, AgentType, initialize_agent, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_mistralai.chat_models import ChatMistralAI

In [ ]:
load_dotenv()
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
NVD_API_KEY = os.getenv("NVD_API_KEY")
NVD_API_URL = "https://services.nvd.nist.gov/rest/json/cves/2.0"

In [ ]:
@tool(return_direct=True)
def get_cve_details(cve_id: str):
    """Looks up details from the National Vulnerability Database about a specific CVE by its ID (e.g., CVE-2024-47195)"""
    headers = {"apiKey": NVD_API_KEY}
    params = {"cveId": cve_id}
    response = requests.get(NVD_API_URL, params=params, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if "vulnerabilities" in data and data["vulnerabilities"]:
            cve_data = data["vulnerabilities"][0]["cve"]
            print(cve_data)
            return f"{cve_data['id']}: {cve_data['descriptions'][0]['value']}"
    
    return "CVE not found."

In [ ]:
@tool
def add5(a: int) -> int:
    """Adds 5 to a."""
    return a+5

In [ ]:
@tool
def get_cves_by_date_range(date_range: str) -> str:
    """Searches for CVEs published within a specific date range in YYYY-MM-DD format. Example input: '2024-01-01 2024-01-31'"""
    dates = date_range.split()
    headers = {"apiKey": NVD_API_KEY}
    params = {
        "pubStartDate": f"{dates[0]}T00:00:00.000",
        "pubEndDate": f"{dates[1]}T23:59:59.999"
    }
    
    response = requests.get(NVD_API_URL, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if "vulnerabilities" in data and data["vulnerabilities"]:
            # Format the results into a readable string
            results = []
            for vuln in data["vulnerabilities"]:
                cve = vuln["cve"]
                cve_id = cve["id"]
                description = cve["descriptions"][0]["value"] if cve["descriptions"] else "No description available"
                published = cve.get("published", "Date unknown")
                results.append(f"- {cve_id} (Published: {published}): {description}")
            
            return f"Found {len(results)} Vulnerabilities:\n" + "\n".join(results)
        return "No vulnerabilities found in this date range."
    return f"Error accessing NVD API: {response.status_code}"

In [ ]:
tools = [get_cve_details, get_cves_by_date_range, add5]

In [ ]:
print(get_cve_details("CVE-2024-47195"))

In [ ]:
print(get_cves_by_date_range("2024-01-01 2024-01-31"))

In [ ]:
llm = ChatMistralAI(model="mistral-small", api_key=MISTRAL_API_KEY, temperature=0)
agent=initialize_agent(tools,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [ ]:
# Run the chatbot
while True:
    user_input = input("Ask about a vulnerability or type 'exit': ")
    if user_input == "exit":
        break
    response = agent.invoke(
        {
            "input": (
                "What are the details of CVE-2024-47195?"
            )
        }
    )
    print(response)

In [ ]:
response.tool_calls